In [1]:
# zaujímavá pomôcka 

"""


{} - altgr + B
[] - altgr + F
#  - altgr + X
$  - altgr + ô
^  - altgr + 3


"""

'\n\n\n{} - altgr + B\n[] - altgr + F\n#  - altgr + X\n$  - altgr + ô\n^  - altgr + 3\n\n\n'

## Import of libraries

In [2]:
import os
import math
import time

# coreference resolution libs
import spacy
import neuralcoref

import pandas as pd

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer # web - https://scikit-learn.org/stable/index.html

# Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# graphs
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# images for jupyter
from IPython.display import Image

## spacy and neuralcoref

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
neuralcoref.add_to_pipe(nlp)

## Dataset import

In [5]:
news = pd.read_csv("../data/fake_news/KaiDMML.csv")

## pandas

In [6]:
news

,Unnamed: 0,url,title,content,publication,type,label,json
0,0,http://www.addictinginfo.org/2016/09/19/proof-...,Proof The Mainstream Media Is Manipulating The...,I woke up this morning to find a variation of ...,http://www.addictinginfo.org,BuzzFeed,fake,https://github.com/KaiDMML/FakeNewsNet/raw/old...
1,1,http://eaglerising.com/36899/charity-clinton-f...,Charity: Clinton Foundation Distributed “Water...,Former President Bill Clinton and his Clinton ...,http://eaglerising.com,BuzzFeed,fake,https://github.com/KaiDMML/FakeNewsNet/raw/old...
2,2,http://eaglerising.com/36880/a-hillary-clinton...,A Hillary Clinton Administration May be Entire...,After collapsing just before trying to step in...,http://eaglerising.com,BuzzFeed,fake,https://github.com/KaiDMML/FakeNewsNet/raw/old...
3,3,http://www.addictinginfo.org/2016/09/19/trumps...,Trump’s Latest Campaign Promise May Be His Mos...,"Donald Trump is, well, deplorable. He’s sugges...",http://www.addictinginfo.org,BuzzFeed,fake,https://github.com/KaiDMML/FakeNewsNet/raw/old...
4,4,http://www.proudcons.com/clinton-foundation-ca...,Website is Down For Maintenance,Website is Down For Maintenance,http://www.proudcons.com,BuzzFeed,fake,https://github.com/KaiDMML/FakeNewsNet/raw/old...
...,...,...,...,...,...,...,...,...
402,402,http://politi.co/2csN1WG,"Donald Trump, Germany’s disfavored son – POLITICO","KALLSTADT, Germany — Few places in Germany are...",http://politi.co,PolitiFact,real,https://github.com/KaiDMML/FakeNewsNet/raw/old...
403,403,http://newsbake.com/entertainment-news/tv/brea...,BREAKING: Hollywood Legend Just Died Of Terrib...,Hollywood loses yet another one of their deare...,http://newsbake.com,PolitiFact,real,https://github.com/KaiDMML/FakeNewsNet/raw/old...
404,404,http://politi.co/2d1qa5t,Worst. President. Ever.,"As my 25th wedding anniversary approached, I t...",http://politi.co,PolitiFact,real,https://github.com/KaiDMML/FakeNewsNet/raw/old...
405,405,http://cnn.it/2dh5vq9,Don King drops N-word while introducing Donald...,Story highlights Trump was sitting in a chair ...,http://cnn.it,PolitiFact,real,https://github.com/KaiDMML/FakeNewsNet/raw/old...


In [7]:
news.count()

Unnamed: 0     407
url            368
title          407
content        405
publication    368
type           407
label          407
json           407
dtype: int64

In [8]:
news[news['label']=="fake"].count()

Unnamed: 0     210
url            173
title          210
content        208
publication    173
type           210
label          210
json           210
dtype: int64

In [9]:
news[news['label']=="real"].count()

Unnamed: 0     197
url            195
title          197
content        197
publication    195
type           197
label          197
json           197
dtype: int64

### Dataset cleaning

In [10]:
# vymazem vsetky nove riadky
news = news.replace(r'\n',' ', regex=True)
news = news.replace(r'\r',' ', regex=True)

In [11]:
# Find all NaN cells

col_name = "content"
np.where(pd.isnull(news[col_name]))

(array([211, 236], dtype=int64),)

In [12]:
# Just checking, if it is really empty

news.iloc[211,:]

Unnamed: 0                                                   211
url            http://cnnews3.com/hiv-virus-detected-walmart-...
title          cnnews3.com -&nbspcnnews3 Resources and Inform...
content                                                      NaN
publication                                   http://cnnews3.com
type                                                  PolitiFact
label                                                       fake
json           https://github.com/KaiDMML/FakeNewsNet/raw/old...
Name: 211, dtype: object

In [13]:
# Just checking, if it is really empty

news.iloc[236,:]

Unnamed: 0                                                   236
url            http://uspoln.com/2017/03/17/tom-price-better-...
title                                                       Home
content                                                      NaN
publication                                    http://uspoln.com
type                                                  PolitiFact
label                                                       fake
json           https://github.com/KaiDMML/FakeNewsNet/raw/old...
Name: 236, dtype: object

In [14]:
# Create new dataframe without 2 rows from previous dataframe
update_news = news.drop([211,236])

# Just checking, if it is really 405
update_news.count()

Unnamed: 0     405
url            366
title          405
content        405
publication    366
type           405
label          405
json           405
dtype: int64

In [15]:
update_news.iloc[211,:]

Unnamed: 0                                                   212
url                                                          NaN
title          Executive Order Leads to Capture of ISIS Leade...
content        Terror suspect, Rasheed Muhammad, was arrested...
publication                                                  NaN
type                                                  PolitiFact
label                                                       fake
json           https://github.com/KaiDMML/FakeNewsNet/raw/old...
Name: 212, dtype: object

In [16]:
update_news['content'] = update_news['content'].str.replace('â€™',"\'")
update_news['content'] = update_news['content'].str.replace('â€ś',"“")

In [17]:
update_news.iloc[34,3]

'0 SHARES Facebook Twitter  Bernard Sansaricq, former president of the Haitian Senate, issued a blistering statement condemning the Clinton Foundation, which has been posted at Donald Trump’s campaign website.  Sansaricq’s statement says:  Sadly, when an earthquake rocked the nation of Haiti in 2010, corruption moved in faster than the help so desperately needed. Today, the people of Haiti are still suffering despite the billions of dollars that have flowed into the Clinton Foundation.  The Clintons exploited this terrible disaster to steal billions of dollars from the sick and starving people of Haiti. The world trusted the Clintons to help the Haitian people during their most desperate time of need and they were deceived.  The Clintons and their friends are richer today while millions still live in tents. The world deserves to know where the money went and why help was never sent.  The situation Sansaricq refers to is discussed in detail here, and also features prominently in Peter S

In [28]:
update_news.count()

Unnamed: 0     405
url            366
title          405
content        405
publication    366
type           405
label          405
json           405
dtype: int64

In [89]:
real = pd.DataFrame()
fake = pd.DataFrame()

real = update_news[update_news['label']=="real"]
fake = update_news[update_news['label']=="fake"]

In [102]:
real['content'].str.split().str.len().mean()

real_news = pd.DataFrame()

real_news['content'] = real['content'].str.replace('[^\w\s]','')
realpocet = real_news['content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
realpocet.tolist().count(1)

C:\Users\kstef\miniconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


6920

In [103]:
fake['content'].str.split().str.len().max()

fake_news = pd.DataFrame()

fake_news['content'] = fake['content'].str.replace('[^\w\s]','')
fakepocet = fake_news['content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
fakepocet.tolist().count(1)

C:\Users\kstef\miniconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


5728

In [88]:
# pocet vsetkych slov v zazname
update_news['content'].str.split().str.len().sum()

212299

In [37]:
# priemerny pocet slov na zaznam
update_news['content'].str.split().str.len().mean()

524.1950617283951

In [38]:
# minimalny pocet slov na zaznam
update_news['content'].str.split().str.len().min()

5

In [39]:
# maximalny pocet slov na zaznam
update_news['content'].str.split().str.len().max()

5459

In [85]:
# unikatny pocet slov

In [60]:
updated_news = pd.DataFrame()

updated_news['content'] = update_news['content'].str.replace('[^\w\s]','')
updated_news['word_count'] = updated_news['content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

C:\Users\kstef\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
pocet = updated_news['content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

In [84]:
pocet.tolist().count(1)

9224

In [29]:
update_news[update_news['label']=="fake"].count()

Unnamed: 0     208
url            171
title          208
content        208
publication    171
type           208
label          208
json           208
dtype: int64

In [30]:
update_news[update_news['label']=="real"].count()

Unnamed: 0     197
url            195
title          197
content        197
publication    195
type           197
label          197
json           197
dtype: int64

## Coreference resolution via Neuralcoref library

In [18]:
# udaje bez zamen budu umiestnene v liste, neskor sa daju do stlpca

# zadefinujem si teda list
coref_list = []


# vytvorim si dataframe, kam si skopirujem z vycisteneho dataframe-u stlpce title, label, content
coref_news = update_news[['label', 'content']].copy()
coref_news.count()


# zadefinujem si dlzku dataframe-u
df_len = coref_news.shape[0]

In [19]:
mapping = {coref_news.columns[0]:'label', coref_news.columns[1]: 'without_coreference'}
coref_news = coref_news.rename(columns=mapping)

In [20]:
print(df_len)

405


In [21]:
coref_news.head()

,label,without_coreference
0,fake,I woke up this morning to find a variation of ...
1,fake,Former President Bill Clinton and his Clinton ...
2,fake,After collapsing just before trying to step in...
3,fake,"Donald Trump is, well, deplorable. He’s sugges..."
4,fake,Website is Down For Maintenance


In [22]:
# zaujima nas, ci je content typu string, ak ano, mozeme ho koreferovat

print(type(coref_news.loc[0, "without_coreference"]))

<class 'str'>


#### ... coreference > in progress

In [23]:
def dopln_zamena(a, b):
    start_time = time.time()
    for x in range(a, b):
        print("Coreferencing cell number: %d ..." %(x), end='')
        
        # coreferencing
        ready = coref_news.iloc[x, 1]
        text = nlp(ready)
        coref_list.append(text._.coref_resolved)
        
        print(" DONE.\n")

In [24]:
%%time
dopln_zamena(0, df_len)

Coreferencing cell number: 0 ... DONE.

Coreferencing cell number: 1 ... DONE.

Coreferencing cell number: 2 ... DONE.

Coreferencing cell number: 3 ... DONE.

Coreferencing cell number: 4 ... DONE.

Coreferencing cell number: 5 ... DONE.

Coreferencing cell number: 6 ... DONE.

Coreferencing cell number: 7 ... DONE.

Coreferencing cell number: 8 ... DONE.

Coreferencing cell number: 9 ... DONE.

Coreferencing cell number: 10 ... DONE.

Coreferencing cell number: 11 ... DONE.

Coreferencing cell number: 12 ... DONE.

Coreferencing cell number: 13 ... DONE.

Coreferencing cell number: 14 ... DONE.

Coreferencing cell number: 15 ... DONE.

Coreferencing cell number: 16 ... DONE.

Coreferencing cell number: 17 ... DONE.

Coreferencing cell number: 18 ... DONE.

Coreferencing cell number: 19 ... DONE.

Coreferencing cell number: 20 ... DONE.

Coreferencing cell number: 21 ... DONE.

Coreferencing cell number: 22 ... DONE.

Coreferencing cell number: 23 ... DONE.

Coreferencing cell number:

KeyboardInterrupt: 

#### ... coreference > done

In [25]:
print(type(coref_list))

<class 'list'>


In [26]:
# klasicky
print(coref_news.iloc[211,1])

Terror suspect, Rasheed Muhammad, was arrested on Tuesday, January 30, at approximately 1:32AM EST at John F. Kennedy International Airport. This marks the first successful story following President Trump’s executive order to protect the nation from foreign terrorist entry into the United States. Muhammad, 32, was questioned due to the heightened security measures that resulted from the presidential executive order. The suspect attempted to enter the country with a tourist visa and claimed to be visiting family in order to attend this year’s Super Bowl LI.    Former Attorney General, Sally Yates, who was ousted by President Trump after failing to support the executive order, released a public apology via popular social media, Snapchat.    “I would like to express a sincere and utter apology to President Donald J. Trump. Due to unforeseen circumstances, there is no way I could have predicted the outcome of the situation. If afforded the opportunity to continue my position as Attorney Ge

In [27]:
# koreferencovany
print(coref_list[211])

IndexError: list index out of range

In [ ]:
# musi byt 405

print(len(coref_list))

In [ ]:
# vlozim vytvoreny list ako novy stlpec

coref_news['with_coreference'] = coref_list

In [ ]:
coref_news.count()

In [ ]:
coref_news.to_csv('coref_news.csv', sep=';')